In [ ]:
## Library Imports
print("Loading Libraries")

# Standard Libraries 
import pandas as pd
import numpy as np
import re, string
import os
from collections import OrderedDict
from datetime import datetime, timedelta, timezone
import pytz

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

# MongoDB
from pymongo import MongoClient

# NLP Libraries
import textacy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# Timezone Info - TO DO - Fix Docker container for matching timezone
local_tz = pytz.timezone('America/Los_Angeles')
os.environ['TZ'] = 'America/Los_Angeles'

print("Loading defs")

def utc_to_local(utc_dt):
    local_dt = utc_dt.replace(tzinfo=pytz.utc).astimezone(local_tz)
    return local_tz.normalize(local_dt)

utc_to_local(datetime.now())

## Import Parameters from Topic_Extractor_Params.dat




## Assign Params from .dats ##
topic_params =    param_loader('Topic_Extractor_Params.dat')
mongo_params =    param_loader('../Scraper_Params.dat')

## Stop Words ##
stop_words_list = stop_loader('Stop_Words_List.dat')

# ## Mongo Params ##
# mongo_host        = str(mongo_params['mongodb_host'])
# mongo_port        = int(mongo_params['mongodb_port'])
# mongo_db          = str(mongo_params['db'])
# mongo_collection  = str(mongo_params['collection'])

# ## Query Params ##
# n_hours           = int(topic_params['Time Frame in Hours'])
# red_or_blue       = str(topic_params['Red, Blue or All'])

# ## Cleaner Params ##
# fix_unicode       = bool(int(topic_params['Fix Unicode']))
# lowercase         = bool(int(topic_params['All Lowercase']))
# no_accents        = bool(int(topic_params['Remove Accents']))
# no_contradictions = bool(int(topic_params['Remove Contradictions']))
# no_emails         = bool(int(topic_params['Remove Emails']))
# no_newline        = bool(int(topic_params['Remove Newline']))
# no_punctuation    = bool(int(topic_params['Remove Punctuation']))
# no_currency       = bool(int(topic_params['Replace Currency']))

# ## Spacy Params ##
# spacy_model       = str(topic_params['Spacy Model'])
# min_word_length   = int(topic_params['Min Word Length'])
# use_cleaned       = bool(int(topic_params['Use Cleaned Text']))
# split_columns     = bool(int(topic_params['Split Columns']))
# named_entities    = topic_params['Named Entity List'].split(' ')

# ## TFIDF Params ## 
# if bool(int(topic_params['Use IDF'])):
#     weighting     = 'tfidf'
# else:
#     weighting     = 'tf'
    
# if topic_params['Max Terms'] == '0':
#     max_n_terms   = None
# else:
#     max_n_terms   = int(topic_params['Max Terms'])
    
# column            = str(topic_params['Column to Vectorize'])
# normalize         = bool(int(topic_params['Normalize']))
# sublinear_tf      = bool(int(topic_params['Sublinear TF']))
# smooth_idf        = bool(int(topic_params['Smooth IDF']))
# vocabulary        = topic_params['Vocabulary']
# min_df            = float(topic_params['Min DF'])
# max_df            = float(topic_params['Max DF'])
# min_ic            = float(topic_params['Min IC'])

# ## Decomposition Params ##
# n_topics          = int(topic_params['Number of Topics'])
# model_type        = str(topic_params['Model Type'])

# ## Visualization Params ##
# top_n_terms       = topic_params['Top Terms Per Topic']
# sort_terms_by     = topic_params['Sort Terms By']
# term_depth        = topic_params['Depth of Termite Plot']
# highlight         = topic_params['Highlight']
# directory         = topic_params['Save Directory']

# try:
#     save          = bool(int(topic_params['Save']))
# except:
#     save          = str(topic_params['Save'])

def stop_fixer(file, upper=True, no_punct=True, fix_in_place=False):
    
    '''Reads and parses *file* - Returns list of stops or (if fix_in_place) overwrites existing *file*
    (if upper) adds capitlized version of stop words
    (if no_punct) adds unpunctuated version of stop words'''
    
    vocab_dict = stop_loader('Stop_Words_List.dat', as_list = False)
    
    for key in vocab_dict:
        if no_punct or upper:
            for word in vocab_dict[key]:
                if no_punct:
                    word_punct = re.sub('[^A-Za-z0-9]+', '', word)
                    if word_punct not in vocab_dict[key]:
                        vocab_dict[key].append(word_punct)
                
                if upper:
                    word_caps = word.capitalize()
                    if word_caps not in vocab_dict[key]:
                        vocab_dict[key].append(word_caps)
    
       
        sorted(vocab_dict[key], key=str.lower)
        vocab_dict[key] = sorted(vocab_dict[key], key=str.lower)
        
    if fix_in_place:
        
        writer = open(file, mode='w+')
        
        for key in vocab_dict:
            writer.writelines('\n' + key + '\n')
            for word in vocab_dict[key]:
                writer.writelines(word  + '\n')
    else:        
        return vocab_dict

def spacy_stopword_adder():
    print(f"Adding {len(stop_words_list)} custom stop words to Spacy Model {spacy_model}.")
    
    if not 'nlp' in globals():
        print(f"Loading Spacy Model {spacy_model}.  This could take a while...")
        global nlp
        nlp = spacy.load(spacy_model)
        print("Complete")
    
    for stopword in stop_words_list:
        STOP_WORDS.add(stopword)
        
    nlp.vocab.add_flag(lambda s: s.lower() in spacy.lang.en.stop_words.STOP_WORDS, spacy.attrs.IS_STOP)
    print(f"Complete. There are {len(STOP_WORDS)} stop words in the list.")
    

def mongo_query(host=mongo_host, port=mongo_port, mongo_db=mongo_db, mongo_collection=mongo_collection, 
                articles=True, custom_query=None, n_hours = n_hours, red_or_blue = red_or_blue, ALL=False):

    if custom_query and ALL:
        raise ValueError("Cannot have custom query and ALL query at once.")
    
    client = MongoClient(host=mongo_host, port=mongo_port)
    
    db = getattr(client, mongo_db)
    
    collection = getattr(db, mongo_collection)
    
    if ALL:
        df = pd.DataFrame(list(collection.find()))
        return df
    
    if custom_query:
        df = pd.DataFrame(list(collection.find(custom_query)))
        return df
    
    if articles:
        post = 'articles'
    else:
        post = 'documents'
    
    query = {'is article': articles}
    
    if n_hours == 0:
        print(f"Pulling all articles from {red_or_blue} targets.")
    else:
        print(f"Pulling {red_or_blue} articles from last {n_hours} hours.")
        dt = datetime.utcnow() - timedelta(hours=n_hours)
        query['date'] = {'$gt': dt}
    
    if red_or_blue == 'Red':
        query['target'] = True
    elif red_or_blue == 'Blue':
        query['target'] = False
    else:
        pass
    
    df = pd.DataFrame(list(collection.find(query)))
    
    print(f'''Completed pulling {len(df)} {post}.
    Latest article is from {collection.find_one(sort=[('date', -1)])['date']} UTC''')
    
    return df

def pre_processor(column = 'text'):
    
    cleaned = [] 
    for x in range(len(df)):
        text = textacy.preprocess.preprocess_text(df[column][x],
                                            fix_unicode=fix_unicode,
                                            lowercase=lowercase,
                                            no_punct=no_punctuation,
                                            no_contractions=no_contradictions,
                                            no_currency_symbols=no_currency,
                                            no_emails=no_emails,
                                            no_accents=no_accents)
        if no_newline:
            text = text.replace('\n', ' ') 
        
        cleaned.append(text)
    
    return cleaned

def spacy_processing(spacy_model=spacy_model, use_cleaned = use_cleaned, split_columns = split_columns):
    
    if 'nlp' not in globals():
        print(f"Loading Spacy Model {spacy_model}.  This could take a while...")
        global nlp
        nlp = spacy.load(spacy_model)
        print("Complete")
    
    if use_cleaned:
        corpus    = 'cleaned'
        df_column = 'cleaned'
    else:
        corpus    = 'raw'
        df_column = 'text'   
    
    print(f'''Filtering stops and words shorter than {min_word_length + 1} letters. 
Chunking and identifying {named_entities} entities from {corpus} corpus.''')    
    
    chunks_list = []
    ents_list   = []
    
    for text in df[df_column]:
        
        doc = nlp(str(text))
        
        chunks = []
        ents   = []
        
        for span in doc.noun_chunks:
            if len(span) == 1:
                if span[0].is_stop or len(span[0]) <= min_word_length:
                    continue
                else:
                    chunks.append(span.text)
                    continue
            else:
                chunks.append(span.text)
        
        for ent in doc.ents:
            if ent.label_ in named_entities:
                ents.append(ent.text)
                
        chunks_list.append(chunks)
        ents_list.append(ents)
    
    if split_columns:
        df['chunks'], df['ents'] = chunks_list, ents_list
        del chunks_list, ents_list
        print(f'''Done inserting {len(df.chunks.sum())} chunks and
              {len(df.ents.sum())} entities into df.chunks and df.ents.''')
    else:
        joined_list = [a + b for a, b in zip(chunks_list, ents_list)]
        del chunks_list, ents_list
        df['chunks_ents'] = joined_list
        del joined_list
        print(f"Done inserting {len(df.chunks_ents.sum())} chunks and entities into df.chunks_ents.)")

def topic_modeler(save=True, visualize=True, **kwargs):    
    
    print(f"Loading Vectorizer Paramaters")
    
    vocabulary=None
    
    vectorizer = textacy.vsm.Vectorizer(weighting, normalize, sublinear_tf,
                                        smooth_idf, vocabulary, min_df, max_df,
                                        min_ic, max_n_terms)
    
    print(f"Fitting Vectorizor on {column} column")
    
    doc_term_matrix = vectorizer.fit_transform(df[column])
    
    model = textacy.tm.TopicModel(model_type, n_topics=n_topics)
    model.fit(doc_term_matrix)
    
    topics_string = ""
    
    for topic_idx, top_terms in model.top_topic_terms(vectorizer.id_to_term, top_n=top_n_terms):
        topics_string += ' - '.join(top_terms) + ' '
        print('topic', topic_idx, ':', '   '.join(top_terms))
        
    if visualize:
        if isinstance(save, str):
            save = save
        elif save is True:
            time_stamp = str(utc_to_local(datetime.now()).strftime('%m_%d_%y_%H_%M'))
            save = f"{directory}/{n_hours}hr_{red_or_blue}_{column}_{time_stamp}"
        
        model.termite_plot(doc_term_matrix, vectorizer.id_to_term, highlight_topics=highlight,
                       topics=-1,  n_terms=term_depth, sort_terms_by=sort_terms_by, save=save)

## Import Database from Mongo
df = mongo_query()

## Preprocessing
df['cleaned'] = pre_processor()

## Adding Stop Words to spacy
spacy_stopword_adder()

## Spacy Tokenization and Entity Recognition
spacy_processing()

## Run Topic Modeler and Visualize Results
topic_modeler()

In [5]:
!pip install textacy

    100% |████████████████████████████████| 143kB 2.3MB/s ta 0:00:01
    100% |████████████████████████████████| 2.0MB 1.9MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 2.8MB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 3.1MB/s ta 0:00:011
    100% |████████████████████████████████| 17.6MB 144kB/s eta 0:00:01   13% |████▎                           | 2.4MB 7.1MB/s eta 0:00:03    27% |████████▊                       | 4.8MB 4.8MB/s eta 0:00:03    64% |████████████████████▋           | 11.3MB 8.1MB/s eta 0:00:01    65% |████████████████████▉           | 11.4MB 880kB/s eta 0:00:07    73% |███████████████████████▊        | 13.0MB 4.3MB/s eta 0:00:02
    100% |████████████████████████████████| 92kB 3.6MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 1.3MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 2.4MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 2.7MB/s ta 0:00:01
    100% |█████████████████████████

  Running setup.py bdist_wheel for python-levenshtein ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
  Running setup.py bdist_wheel for spacy ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/fb/00/28/75c85d5135e7d9a100639137d1847d41e914ed16c962d467e4
  Running setup.py bdist_wheel for pyemd ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/e4/ba/b0/1f4178a35c916b22fc51dc56f278125d4b8cfb0592e5f0cc24
  Running setup.py bdist_wheel for ftfy ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/37/54/00/d320239bfc8aad1455314f302dd82a75253fc585e17b81704e
  Running setup.py bdist_wheel for murmurhash ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/b8/94/a4/f69f8664cdc1098603df44771b7fec5fd1b3d8364cdd83f512
  Running setup.py bdist_wheel for cymem ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/55/8d/4a/f6328252aa2aaec0b1cb906fd96a1566d77f0f67701071ad13

In [2]:
import os

In [1]:
from dat import defaults

from ast import literal_eval
from collections import namedtuple, OrderedDict
from pathlib import Path

class parameters:

    def __init__(self, stop=True, from_file=True, verbose=False):
        
        import dat
        
        self.defaults   = dat.defaults
        self.params     = self.param_loader(self.defaults['PARAMS']['Default Path'])
        self.mongo      = self.param_loader(self.defaults['MONGO']['Default Path'])
        
            
    def param_writer(self, file, params, default_location= True, overwrite=False):
        '''A writer definition for writing new param files and overwriting old ones.  USE WITH CAUTION!'''
        
        from pathlib import Path
        
        if Path(file).is_file():
            overwrite = literal_eval(input(f'{file} already exists. Overwrite?  :'))
            if overwrite:
                pass
            else:
                file = input('/path/to/new.file :')

        writer = open(file, mode='w+')
        
        for key in params:
            writer.writelines('\n' + '{}'.format(key) + '\n')
            for value in params[key]:
                
                writer.writelines('{:30}{}{}'.format(value, '= ', params[key][value]) + '\n')
        print(f"Completed writing {list(params.keys())} to '{file}'.")
    
    def param_loader(self, file):
        '''A loader definition for loading script dependant parameters from file'''
        
        self.file = file
        
        from pathlib import Path
        
        if not Path(file).is_file():
            response = literal_eval(input(f'"{file}" not found.  Would you like to attempt to generate a default?'))
            if response:
                for key in self.dd:
                    if self.dd[key]['Default Path'] == file:
                        response = literal_eval(input(f'Found Default "{file}". Generate?'))
                        if response:
                            dft_dict = self.dd[key]['DAT']
                            self.param_writer(file, dft_dict)
            
            else:
                pass
        
        fileObj = open(file, mode='r')
            
        params_dict  = OrderedDict()
        
        for line in fileObj:
            if not line:
                pass
            line = line.strip()
            key_value = line.split('=')
    
            if "#" in line:
                section = line[3:].replace(" ", "")
                params_dict[section] = {}

            if len(key_value) == 2:
                key = key_value[0].strip().replace(" ", "")
                value = key_value[1].strip()
 
                if not value:
                    value = input(f'Required parameter "{key_value[0].strip()}" is missing: ')
                    while not value:
                        value = input(f'You must enter a value for {key_value[0].strip()}: ')
                    
                
                if "," in value:
                    try:
                        params_dict[section][key] = list(literal_eval(value))
                    except:
                        params_dict[section][key] = value.split(", ")
                else:
                    try:
                        params_dict[section][key] = literal_eval(value)
                    except:
                        params_dict[section][key] = value
        
        param_nt = namedtuple('params', params_dict.keys())(**params_dict)
        return param_nt

ModuleNotFoundError: No module named 'dat'

In [9]:
parameters()

ModuleNotFoundError: No module named 'dat'

In [6]:
## Library Imports
print("Loading Libraries")

# Standard Libraries 
import pandas as pd
import numpy as np
import re, string
import pprint
import os
from collections import OrderedDict
from datetime import datetime, timedelta, timezone
import pytz
from ast import literal_eval
from collections import namedtuple, OrderedDict

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

# MongoDB
from pymongo import MongoClient

# NLP Libraries
import textacy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS


class Politiprocess:

    model_params   = 'Topic_Extractor_Params.dat'
    mongo_params   = '../Scraper_Params.dat'
    
    stop_list = []
    
    def __init__(self, from_file=True, verbose=False):
        print("Loading Params")
        
        self.params = self.param_loader(self.model_params)
        self.mongo  = self.param_loader(self.mongo_params)
        
        if verbose:
            print(self.params)
    
    def param_loader(self, file):
        '''A loader definition for loading script dependant parameters from file'''
        
        fileObj = open(file, mode='r')
        params_dict  = OrderedDict()
        
        for line in fileObj:
            if not line:
                pass
            line = line.strip()
            key_value = line.split('=')
    
            if "#" in line:
                section = line[3:].replace(" ", "")
                params_dict[section] = {}

            if len(key_value) == 2:
                key = key_value[0].strip().replace(" ", "")
                value = key_value[1].strip()
 
                if not value:
                    value = input(f'Required parameter "{key_value[0].strip()}" is missing: ')
                    while not value:
                        value = input(f'You must enter a value for {key_value[0].strip()}: ')
                    
                
                if "," in value:
                    try:
                        params_dict[section][key] = list(literal_eval(value))
                    except:
                        params_dict[section][key] = value.split(", ")
                else:
                    try:
                        params_dict[section][key] = literal_eval(value)
                    except:
                        params_dict[section][key] = value
        
        param_nt = namedtuple('params', params_dict.keys())(**params_dict)
        return param_nt
    
    def stop_loader(self, file='Stop_Words_List.dat', as_list=True):
        '''A loader for loading a unicode file of stop words as dictionaries.
        Keys are categories and Values are lists of terms'''
        
        fileObj = open(file, mode='r')
        
        vocab_dict = {}
        vocab_list = []
        current_key = ""
        
        for line in fileObj.read().split('\n'):
            
            if not line:
                continue
                
            if "#" in line:
                vocab_list = []
                vocab_dict[line] = []
                current_key = str(line)
            
            if line[0].isalpha():
                vocab_list.append(line)
            
            vocab_dict[current_key] = vocab_list
            
        if as_list:
            vocab_list = []
            for terms in vocab_dict.values():
                vocab_list.extend(terms)
            print(f"Loading {len(vocab_list)} stop words from {list(vocab_dict.keys())}")
            return vocab_list
        
        return vocab_dict
    
    def stop_fixer(self, file='Stop_Words_List.dat', upper=True, no_punct=True, fix_in_place=False):
        '''Reads and parses *file* - Returns list of stops or (if fix_in_place) overwrites existing *file*
        (if upper) adds capitlized version of stop words
        (if no_punct) adds unpunctuated version of stop words'''
        
        vocab_dict = self.stop_loader('Stop_Words_List.dat', as_list = False)
        
        for key in vocab_dict:
            if no_punct or upper:
                for word in vocab_dict[key]:
                    if no_punct:
                        word_punct = re.sub('[^A-Za-z0-9]+', '', word)
                        if word_punct not in vocab_dict[key]:
                            vocab_dict[key].append(word_punct)
                    
                    if upper:
                        word_caps = word.capitalize()
                        if word_caps not in vocab_dict[key]:
                            vocab_dict[key].append(word_caps)
        
           
            sorted(vocab_dict[key], key=str.lower)
            vocab_dict[key] = sorted(vocab_dict[key], key=str.lower)
            
        if fix_in_place:
            
            writer = open(file, mode='w+')
            
            for key in vocab_dict:
                writer.writelines('\n' + key + '\n')
                for word in vocab_dict[key]:
                    writer.writelines(word  + '\n')
        else:        
            return vocab_dict


    
#     def mongo_query(host=mongo_host, port=mongo_port, mongo_db=mongo_db, mongo_collection=mongo_collection, 
#                     articles=True, custom_query=None, n_hours = n_hours, red_or_blue = red_or_blue, ALL=False):
    
#         if custom_query and ALL:
#             raise ValueError("Cannot have custom query and ALL.")
        
#         client = MongoClient(host=mongo_host, port=mongo_port)
        
#         db = getattr(client, mongo_db)
        
#         collection = getattr(db, mongo_collection)
        
#         if ALL:
#             df = pd.DataFrame(list(collection.find()))
#             return df
        
#         if custom_query:
#             df = pd.DataFrame(list(collection.find(custom_query)))
#             return df
        
#         if articles:
#             post = 'articles'
#         else:
#             post = 'documents'
        
#         query = {'is article': articles}
        
#         if n_hours == 0:
#             print(f"Pulling all articles from {red_or_blue} targets.")
#         else:
#             print(f"Pulling {red_or_blue} articles from last {n_hours} hours.")
#             dt = datetime.utcnow() - timedelta(hours=n_hours)
#             query['date'] = {'$gt': dt}
        
#         if red_or_blue == 'Red':
#             query['target'] = True
#         elif red_or_blue == 'Blue':
#             query['target'] = False
#         else:
#             pass
        
#         df = pd.DataFrame(list(collection.find(query)))
        
#         print(f'''Completed pulling {len(df)} {post}.
#         Latest article is from {collection.find_one(sort=[('date', -1)])['date']} UTC''')
        
#         return df

Loading Libraries


NameError: name 'mongo_host' is not defined

In [ ]:
pp = Politiprocess()

In [ ]:
from wordcloud import WordCloud

In [ ]:
topics_string

In [ ]:
wordcloud = WordCloud().generate(topics_string)

In [ ]:
wordcloud = WordCloud(max_font_size=80, width = 800, height=400).generate(topics_string)
plt.figure(figsize=(12,6))
plt.imshow(wordcloud, interpolation="bicubic")
plt.axis("off")
plt.show()